In [ ]:
# predict_live_cpu.py
# RUN THIS IN YOUR "asl_cpu_env"
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import mediapipe as mp # Uses the OLD solutions API
from collections import deque
import time

# --- 1. DEFINE CONSTANTS ---
ASL_MODEL_PATH = 'models/asl_landmark_model.keras' # Your landmark model
class_names = [
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 
    'del', 'nothing', 'space'
]
ZERO_VECTOR_NP = np.array([0.0] * 63, dtype=np.float32).reshape(1, -1)

# --- 2. LOAD YOUR CLASSIFIER MODEL ---
try:
    asl_model = load_model(ASL_MODEL_PATH)
    print("ASL Landmark model loaded successfully!")
except Exception as e:
    print(f"Error loading ASL model: {e}")
    exit()

# --- 3. INITIALIZE MEDIAPIPE (OLD API) ---
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False, 
    max_num_hands=1,
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe (Old API) Hand Detector created.")

# --- 4. SET UP WEBCAM ---
print("Opening camera...")
cap = cv2.VideoCapture(0, cv2.CAP_V4L2) 
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P','G'))
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()
print("Camera opened successfully.")

# --- 5. PREDICTION & SMOOTHING VARIABLES ---
predictions_deque = deque(maxlen=10) # Holds the last 10 predictions
current_label = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)
    
    # --- 6. MEDIAPIPE HAND DETECTION ---
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    model_input = ZERO_VECTOR_NP
    
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        
        # --- 7. NORMALIZE LANDMARKS (Same as in training) ---
        wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
        landmark_vector = []
        for landmark in hand_landmarks.landmark:
            landmark_vector.append(landmark.x - wrist.x)
            landmark_vector.append(landmark.y - wrist.y)
            landmark_vector.append(landmark.z - wrist.z)
        
        model_input = np.array(landmark_vector, dtype=np.float32).reshape(1, -1)
        
        # Draw skeleton on the frame
        mp_drawing.draw_landmarks(
            frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
    # --- 8. MAKE A PREDICTION (ALWAYS) ---
    prediction = asl_model.predict(model_input, verbose=0)
    pred_index = np.argmax(prediction[0])
    
    # --- 9. SMOOTH THE PREDICTION ---
    predictions_deque.append(pred_index)
    
    if len(predictions_deque) == 10:
        most_common_pred = np.bincount(predictions_deque).argmax()
        confidence = prediction[0][most_common_pred]
        
        if confidence > 0.6: # 60% confidence
            current_label = class_names[most_common_pred]
        else:
            if class_names[most_common_pred] == 'nothing':
                 current_label = "nothing"
            else:
                 current_label = "..."
    else:
        current_label = "..."

    cv2.putText(
        frame, current_label, (20, 40), 
        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2
    )
    cv2.imshow('ASL CPU Prediction - Press "q" to quit', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 10. CLEAN UP ---
print("Cleaning up and closing...")
hands.close()
cap.release()
cv2.destroyAllWindows()

2025-11-12 13:07:13.826326: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 13:07:14.050965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762952834.138641    1057 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762952834.162954    1057 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762952834.340478    1057 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

ASL Landmark model loaded successfully!
MediaPipe (Old API) Hand Detector created.
Opening camera...
Camera opened successfully.


I0000 00:00:1762952840.380544    1057 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1762952840.429228    1250 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: llvmpipe (LLVM 20.1.2, 256 bits)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1762952840.481264    1202 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762952840.516328    1212 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1762952841.338634    1165 service.cc:152] XLA service 0x7b9af8003ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762952841.338664    1165 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.

Cleaning up and closing...
